In [72]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException

from PIL import Image
import io

import time

import pandas as pd

**ブラウザの起動**

**各ページのURLを取得する関数**
1. SUUMOの物件情報(indexページ)は、基本的に内部リンク（'https:~がない"）形式でHTMLが作成されているため、内部リンクのみ全体のページから取得する
2. 取得したURLをリスト形式にまとめる
3. URLのリストを返す関数を作成する

In [2]:
def get_urls(soup) -> list:
    """
    一覧のページからURL個別ページのURLを取得する関数
    """
    h2_elems = soup.find_all('h2', attrs={'class' : 'property_unit-title'})
    
    urls = []
    
    for h2_elem in h2_elems:
        a_elem = h2_elem.find('a')
        url = a_elem.attrs['href']
        
        urls.append(url)
        
    return urls

**各ページの物件情報を収集する関数**
- 入力は各ページの内部リンクのURL（シングル）
- 出力は各ページのsoup

In [3]:
def get_page_soup(internal_url:str):
    """
    各物件へのリンクを作成して、アクセスして、各ページのsoupを出力する
    """
    # ページ内リンクをドメインと結びつけて直にアクセスできるリンクに変換している
    page_url = 'https://suumo.jp' + internal_url
    page_res = requests.get(page_url)
    page_soup = BeautifulSoup(page_res.content, 'html.parser')
    
    return page_soup

**各ページの物件情報を収集する関数**
- 入力は各ページのsoup
- 出力は各ページの物件情報が格納された辞書

In [4]:
def get_house_details(page_soup) -> dict:
    """[summary]　
    　その後に各物件のページから「物件情報」にアクセスして、「物件情報」のテーブルまるごと取得する。
    Args
        internal_url[str]:SUUMOの各物件の内部リンク
    """
    #物件詳細のページへのリンクの取得
    house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline2'})
    
    if house_details_a_elem is None:
        house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline'})
        
    house_details_url = house_details_a_elem.attrs['href']
    
    #物件詳細のページへのアクセス    
    house_details_res = requests.get(house_details_url)
    house_details_soup = BeautifulSoup(house_details_res.content, 'html.parser')
    
    # テーブルの取得
    house_details_info = house_details_soup.find('table', {'class': 'pCell10'})

    return house_details_info

**各ページの物件情報を収集する関数**
- 入力はtable要素
- 出力はth要素がKey、td要素がValueになった辞書

In [5]:
def extract_table_data(table) -> dict:
    """
    　tableのHTMLからthとtd内の情報を取得する　
    """
    house_details_dict = {}

    for row in table.find_all('tr'):
        data_head = row.find_all('th')
        data_value = row.find_all('td')

        data_head_01 = data_head[0].text.replace('\n', '').replace('ヒント', '')
        data_value_01 = data_value[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

        try:
            if data_head[1] is not None:
                data_head_02 = data_head[1].text.replace('\n', '').replace('ヒント', '')
                data_value_02 = data_value[1].text.replace('\n', '').replace('\r', '').replace('\t', '').replace('\u3000', '').replace('\xa0', '').replace('[乗り換え案内]', '')

                house_details_dict[data_head_01] = data_value_01
                house_details_dict[data_head_02] = data_value_02
        except IndexError:
            house_details_dict[data_head_01] = data_value_01
            
    return house_details_dict

**各物件のタイトルとコメントを取得する関数**

In [6]:
def get_title_and_comment(page_soup):
    """
    サイト上部にあるタイトルとコメントを取得する関数
    """
    try:
        house_title = page_soup.find('h2', attrs = {'class' : 'fs16'})
        house_comment = page_soup.find('p', attrs = {'class' : 'fs14'})
        house_dict = {'title':house_title.text, 'comment':house_comment.text}
    except:
        house_dict = {'title':'', 'comment':''}
    return house_dict

## テスト用のコードTips

In [7]:
# page_soup = get_page_soup(urls[1]) 
# table = get_house_details(page_soup)
# house_info_dict = extract_table_data(table)
# house_text_dict = get_title_and_comment(page_soup)
# get_house_img(page_soup, 1)
# urls

**家の画像の取得**
- 画像の名前を一意に決めるためにhouse_idを引数に入れている

In [76]:
def get_house_img(page_soup, house_id):
    """
    各ページの写真を取得して、写真をHouseId_IMGIDの形式で保存する。
    """
    print(house_id)
    imgs = page_soup.find_all('img')
    img_list = list()
    img_id = 0
    # 以下で始まるのはIMGタグだがアクセスできないため排除する
    un_img_signal = 'img01.suumo.com'
    
    for img in imgs:
        try:
            print('img', img)
            img_url = img['rel']
            img_tag = img['alt']
        except KeyError:
            img_url = un_img_signal
            img_tag = ""
            print("house_id", house_id, "img_id", img_id, 'img_url', img_url)

        if not un_img_signal in img_url:
            print("house_id", house_id, "img_id", img_id, 'img_url', str(img_url))
            img_name = str(house_id) + '_' + str(img_id)
            img_id += 1
            img = Image.open(io.BytesIO(requests.get(img_url).content))
            img.save(f'imgs/{img_name}.jpg')
            img_list.append({'house_id':house_id, 'img_id':img_id, 'img_tag':img_tag, 'img_name':img_name})
    
    return img_list

**Index1ページ分のURL**

- ここのループでは、Index1ページ分のURLをすべて取ってきている

In [9]:
def get_index_info(urls:list, house_info:list, house_id:int) -> list:
    for url in urls:
        page_soup = get_page_soup(url)
        table = get_house_details(page_soup)
        house_info_dict = extract_table_data(table)
        house_text_dict = get_title_and_comment(page_soup)
        house_img_list = get_house_img(page_soup, house_id)

        house_dict = {'House_ID': house_id, 'text':house_text_dict, 'info':house_info_dict, 'imgs':house_img_list}
        house_info.append(house_dict)
        
        house_id += 1
        
        time.sleep(20)
        
    return house_info, house_id

In [10]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
browser.get(url)

count = 0

url = browser.current_url
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


**すべてのページを見れるように次へのボタンを押す**

- このままでは、すべてのページを自動的にクロールすることは不可能。そこで、すべてのページをクロールできるようにプログラムで次へのボタンを押すことを試みます。

In [ ]:
data_list = []
#house_info = []
house_id = 0

count = 0

url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)

while True:
    url = browser.current_url
    print(url)
    #ブラウザの起動
    options = Options()
    options.add_argument('--headless')
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    browser.get(url)
    
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        #a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})
        urls = get_urls(soup)
        house_info, house_id = get_index_info(urls, data_list, house_id)
        
        browser.find_element_by_link_text('次へ').click()
        if count % 10 == 0:
            print("pages:{0}".format(count))
            print('==============================================')
            time.sleep(60)
        time.sleep(60)
        
        count += 1
        print('count', count)
            
    except NoSuchElementException:
        browser.quit()
        break

print('browser quit')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Driver [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101/chromedriver] found in cache


https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999
0
img <img alt="SUUMO(スーモ)" height="46" src="/edit/assets/suumo/img/logo.png" width="138"/>
house_id 0 img_id 0 img_url img01.suumo.com
img <img alt="現地外観写真" class="js-slideLazy-image js-noContextMenu" rel="https://suumo.jp/front/gazo/bukken/020/N010000/img/074/96091074/96091074_0017.jpg"/>
house_id 0 img_id 0 img_url https://suumo.jp/front/gazo/bukken/020/N010000/img/074/96091074/96091074_0017.jpg
img <img alt="間取り図" class="js-slideLazy-image js-noContextMenu" rel="https://suumo.jp/front/gazo/bukken/020/N010000/img/074/96091074/96091074_0020.jpg"/>
house_id 0 img_id 1 img_url https://suumo.jp/front/gazo/bukken/020/N010000/img/074/96091074/96091074_0020.jpg
img <img alt="浴室" class="js-slideLazy-image js-noContextMenu" rel="https://suumo.jp/front/gazo/bukken/020/N010000/img/074/96091074/96091074_0019.jpg"/>
hous

In [ ]:
#house_info

In [58]:
len(house_info)

335

In [59]:
def edit_house_data(house):
    house_id = house['House_ID']
    house_dict = {
        'title' : house['text']['title'],
        'comment' : house['text']['comment'],
        '販売スケジュール' : house['info']['販売スケジュール'],
        'イベント情報' : house['info']['イベント情報'],
        '所在地' : house['info']['所在地'],
        '交通' : house['info']['交通'],
        '販売戸数' : house['info']['販売戸数'],
        '総戸数' : house['info']['総戸数'],
        '価格' : house['info']['価格'],
        '最多価格帯' : house['info']['最多価格帯'],
        '私道負担・道路' : house['info']['私道負担・道路'],
        '諸費用' : house['info']['諸費用'],
        '間取り' : house['info']['間取り'],
        '建物面積' : house['info']['建物面積'],
        '土地面積' : house['info']['土地面積'],
        '建ぺい率・容積率' : house['info']['建ぺい率・容積率'],
        '完成時期(築年月)' : house['info']['完成時期(築年月)'],
        '入居時期' : house['info']['入居時期'],
        '土地の権利形態' : house['info']['土地の権利形態'],
        '構造・工法' : house['info']['構造・工法'],
        '施工' : house['info']['施工'],
        'リフォーム' : house['info']['リフォーム'],
        '用途地域' : house['info']['用途地域'],
        '地目' : house['info']['地目'],
        'その他制限事項' : house['info']['その他制限事項'],
        'その他概要・特記事項' : house['info']['その他概要・特記事項'],
    }
    
    return house_id, house_dict

def houses_adjustment(data:list, houses_data:dict, house_imgs:list) -> dict:
    for house in data:
        # Pandasで加工しやすいようにKeyが一つの辞書に家の情報を変更する
        house_id, house_dict = edit_house_data(house)
        houses_dict[house_id] = house_dict
        # Houseの画像は変更が必要ないので、そのままリストに追加する
        # ただ、一つづつ取り出して追加する必要があるので、要注意
        img_list += house['imgs']

In [60]:
houses_dict = {}
img_list = []

for house in house_info:
    # Pandasで加工しやすいようにKeyが一つの辞書に家の情報を変更する
    house_id, house_dict = edit_house_data(house)
    houses_dict[house_id] = house_dict
    # Houseの画像は変更が必要ないので、そのままリストに追加する
    # ただ、一つづつ取り出して追加する必要があるので、要注意
    img_list += house['imgs']

In [61]:

#houses_dict
pd.DataFrame(houses_dict).transpose()

,title,comment,販売スケジュール,イベント情報,所在地,交通,販売戸数,総戸数,価格,最多価格帯,...,完成時期(築年月),入居時期,土地の権利形態,構造・工法,施工,リフォーム,用途地域,地目,その他制限事項,その他概要・特記事項
0,令和２年２月末内装リフォーム済み！閑静な住宅地！国道112号線へのアクセス良し！,【リフォーム内容】ユニットバス交換・換気扇取替・トイレ交換・CF貼替・キッチンシンク交換・洗...,-,-,山形県東村山郡中山町大字長崎,ＪＲ左沢線「羽前長崎」歩22分,1戸,1戸,1098万円[ □支払シミュレーション ],-,...,1985年6月,即入居可,所有権,木造2階建,-,2020年2月完了水回り設備交換：キッチン・浴室・トイレ※年月は一番古いリフォーム箇所を表します,１種住居,-,-,設備：公営水道、個別浄化槽、個別LPG、駐車場：カースペース
1,リニューアル工事しました!!,・駐車場拡張・ユニットバス交換・換気扇取替・トイレ交換・CF張替え・かぎ交換・白あり点検・キ...,-,-,山形県東村山郡中山町大字長崎,ＪＲ左沢線「羽前長崎」歩22分,1戸,-,1098万円[ □支払シミュレーション ],-,...,1985年6月,相談,所有権,木造2階建,-,2020年2月完了水回り設備交換：キッチン・浴室・トイレ※年月は一番古いリフォーム箇所を表します,１種住居,-,-,担当者：井上、設備：公営水道、個別浄化槽、個別LPG、駐車場：カースペース
2,◇◇価格見直しました！！◇◇■山形北エリア利便性の良い立地！■■山形市立東小学校まで徒歩７分...,□ときめき通り、医者街通りすずそばで各科クリニックがそろっています！！□□国道13号線へのア...,-,現地見学会（事前に必ずお問い合わせください）日程／公開中,山形県山形市花楯２,ＪＲ仙山線「北山形」歩29分,1戸,1戸,1330万円[ □支払シミュレーション ],-,...,1979年4月,相談,所有権,木造2階建（軸組工法）,(株)ウンノハウス,-,２種住居,-,景観法による規制有,担当者：沼沢 義明、設備：公営水道、本下水、個別LPG、駐車場：カースペース
3,◆オール電化◆■３台以上駐車可能＼ご見学は事前のご予約制です／,◆ご内覧のご予約◆「見学ご予約」をクリックTEL：０８００－８１２－９４０１ 【通話料無料】...,-,現地案内会（事前に必ず予約してください）日程／公開中■現地案内・他地域の建売住宅等ご案内致し...,山形県東根市大字羽入,ＪＲ奥羽本線「神町」歩43分,1戸,1戸,1630万円[ □支払シミュレーション ],-,...,2009年2月,相談,所有権,木造2階建,-,-,無指定,-,-,担当者：小関 芳重、設備：公営水道、本下水、オール電化、駐車場：カースペース
4,瑕疵保険付きの築１６年のリフォーム済住宅！利便性に恵まれた酒田駅まで徒歩約６分の好立地！水廻...,●住まい給付金対象！●瑕疵保険付き住宅！●リフォーム内容・外壁塗装・システムキッチン新品・洗...,-,オープンハウス（事前に必ず予約してください）日程／公開中時間／10:00～17:00完全ご予...,山形県酒田市幸町１,庄内交通「浜田一丁目」歩3分,1戸,1戸,1699万円[ □支払シミュレーション ],-,...,1995年3月,即入居可,所有権,木造2階建（軸組工法）,-,2021年4月完了水回り設備交換：キッチン・浴室・トイレ内装リフォーム：壁・床2021年4月...,１種住居,-,景観法による規制有、準防火地域,担当者：佐藤晴基、設備：公営水道、本下水、都市ガス、駐車場：カースペース
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,インスペクション済 パネルヒーター6台、蓄暖4台、床暖、エアコン4台 エコキュート、ワークス...,インスペクション済 パネルヒーター6台、蓄暖4台、床暖、エアコン4台 エコキュート、ワークス...,-,-,山形県山形市嶋北２,ＪＲ左沢線「東金井」車2.2kmＪＲ奥羽本線「羽前千歳」車2.6km,1戸,-,3990万円[ □支払シミュレーション ],-,...,2009年8月,相談,所有権,木造2階建,鎌田工務店,-,１種低層,-,高さ最高限度有、壁面後退有,設備：公営水道、本下水、駐車場：車庫
331,☆5LDK（主屋）+3LDK（別宅）◆多世帯、店舗利用にもオススメ！◆大手ハウスメーカー施工...,同一敷地内に建っている、主屋と別宅を合わせての販売です。◆主屋は大手ハウスメーカー施工、平成...,-,（事前に必ずお問い合わせください）,山形県山形市大字上東山,ＪＲ仙山線「高瀬」歩28分,1戸,2戸,4500万円[ □支払シミュレーション ],-,...,2017年7月,相談,所有権,木造2階建,住友林業(株),-,都市計画区域外,-,景観法による規制有、高瀬小、高楯中,担当者：小口秀喜、設備：公営水道、本下水、オール電化、駐車場：車庫
332,人気の天童市芳賀タウンの中古住宅！駅まで徒歩７分、イオン天童まで徒歩６分！,■築4年の中古住宅■天童芳賀タウンの大型分譲地内■東南角地で居室は全室南向き！■電動シャッタ...,-,現地案内会（事前に必ず予約してください）日程／公開中時間／10:00～18:00築浅中古住宅！,山形県天童市芳賀タウン南２,ＪＲ奥羽本線「天童南」歩7分,1戸,1戸,5500万円[ □支払シミュレーション ],-,...,2017年6月,即入居可,所有権,木造2階建（軸組工法）,(株)近江建設,-,２種低層,-,高さ最高限度有,担当者：石澤 宏一朗、設備：公営水道、本下水、駐車場：車庫
333,美築の大型住宅です！敷地は圧巻の370坪超え！建物約115坪！一見の価値ある3階建、エレベー...,○おいしい庄内空港まで徒歩28分！○教育施設が近く、お子様の一人での通学も安心の距離。○中学...,-,現地案内会（事前に必ずお問い合わせください）日程／公開中時間／10:00～17:00▼───...,山形県酒田市浜中字上村,ＪＲ羽越本線「酒田」歩99分庄内交通「庄内空港」歩28分,1戸,-,5600万円[ □支払シミュレーション ],-,...,2006年10月,相談,所有権,木造3階建（軸組工法）,-,-,市街化調整区域,-,-,担当者：佐藤 知広、設備：公営水道、個別浄化槽、個別LPG、建築許可理由：都市計画法施行令3...


In [62]:
pd.DataFrame(img_list)

,house_id,img_id,img_tag,img_name
0,0,1,現地外観写真,0_0
1,0,2,間取り図,0_1
2,0,3,浴室,0_2
3,0,4,リビング,0_3
4,0,5,キッチン,0_4
...,...,...,...,...
288,15,20,リビング以外の居室,15_19
289,15,21,スーパー,15_20
290,15,22,小学校,15_21
291,15,23,中学校,15_22
